In [1]:
n_questions_answered = 0
test_filename = 'test_quiz.csv'

In [2]:
def quiz():
    # Adding imports inside function
    from datetime import datetime
    import pandas as pd
    import time
    from ipywidgets import widgets, Layout, Box
    from IPython.display import display, clear_output
    
    quiz_timestamp = datetime.now().strftime('%Y%m%d_%H_%M_%S')
    
    # Importing questions
    global test_filename
    df_questions = pd.read_csv(test_filename)
    n_questions = len(df_questions)

    def create_multipleChoice_widget(category, description, options, correct_answer, img_filepath = None):
        if correct_answer not in options:
            options.append(correct_answer)

        correct_answer_index = options.index(correct_answer)

        alphabet = 'abcdefghijklmnopqrstuvwxyz'
        options_w_letters = [f'{alphabet[i]}) {option}' for i, option in enumerate(options)]

        radio_options = [(words, i) for i, words in enumerate(options_w_letters)]

        alternativ = widgets.RadioButtons(
            options = radio_options,
            description = '',
            disabled = False,
            indent = False,
            align = 'center',
        )

        description_out = widgets.Output(layout=Layout(width='auto'))

        if img_filepath:
            image = open(img_filepath, 'rb').read()
            image_out = widgets.Image(value=image, 
                                      format='png', 
                                      width='200px', 
                                      height='auto',
                                     )

        with description_out:
            print(description)

        feedback_out = widgets.Output()

        def check_selection(b):
            a = int(alternativ.value)
            flag = a==correct_answer_index
            if flag:
                s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
            else:
                correct_answer_letter = alphabet[correct_answer_index]
                s = '\x1b[5;30;41m' + f"incorrect, the correct answer was {correct_answer_letter}) {correct_answer}" + '\x1b[0m' +"\n"
            with feedback_out:
                global n_questions_answered
                n_questions_answered += 1
                print(s)
                s_results = pd.Series({'Category':category, 
                                       'Question':description, 
                                       'Correct Answer':correct_answer, 
                                       'Submitted Answer':options[a], 
                                       'Score':int(flag)})
                quiz_results.append(s_results)
                alternativ.close()
                check.layout.display = 'none'

        check = widgets.Button(description="check")
        check.on_click(check_selection)

        check_out = widgets.HBox([check])

        if img_filepath:
                return widgets.VBox([description_out, 
                             image_out,
                             alternativ, 
                             widgets.HBox([check]), feedback_out], 
                            layout=Layout(display='flex',
                                         flex_flow='column',
                                         align_items='stretch',
                                         width='auto')) 

        return widgets.VBox([description_out, 
                             alternativ, 
                             check_out, 
                             feedback_out], 
                            layout=Layout(
                                display='flex',
                                flex_flow='column',
                                align_items='stretch',
                                width='auto')) 

    def create_questions(df_questions):
        list_widgets = []
        for question_idx, s_question in df_questions.iterrows():
            category = s_question.loc['Category']
            description = s_question.loc['Question']
            options = s_question.loc['Options'].split(', ')
            correct_answer = s_question.loc['Correct Answer']
            image_filename = s_question.loc['Image Filename']

            if '.png' in str(image_filename):
                image_filepath = f'images/{image_filename}'
            else:
                image_filepath = None

            question_widget = create_multipleChoice_widget(category, description, options, correct_answer, image_filepath)
            list_widgets.append(question_widget)
        return list_widgets

    def on_button_clicked(b):
        global n_questions_answered
        if n_questions_answered == n_questions:
            df_final_results = pd.DataFrame(quiz_results)
            final_score = df_final_results['Score'].sum()
            final_score_pct = round(100 * (final_score / float(n_questions)), 2)            
            final_score_message = f'Final Score: {final_score}/{n_questions} ({final_score_pct}%)'
            with final_out:
                print(final_score_message)      
            filename = f'quiz_{quiz_timestamp}.csv'
            filepath = f'quiz_results/{filename}'
            df_final_results.to_csv(filepath)           
            

    list_widgets = create_questions(df_questions)
    quiz_results = []    

    for widget in list_widgets:
        display(widget)
        for child in widget.children:
            if type(child)==widgets.widget_box.HBox:
                child.children[0].on_click(on_button_clicked)
    final_out = widgets.Output()
    display(final_out)    

In [3]:
quiz()

Output()